# Dependencies

In [ ]:
!pip install efficientnet

In [ ]:
import re
import math
import random
import numpy as np 
import pandas as pd 
import os
import shutil
import json
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras.backend as K
from tensorflow.keras import losses
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from efficientnet.keras import EfficientNetB1 as EfficientNet
import gc
print("Using TensorFlow version %s" % tf.__version__)

# Hardware configuration

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# Loading work path

In [ ]:
data_path = KaggleDatasets().get_gcs_path("cassava-leaf-disease-classification")
data_path

# Parameters

In [ ]:
VALIDATION_SIZE = 0.2
BATCH_SIZE = 16 * REPLICAS
ORIGINAL_WIDTH = 800
ORIGINAL_HEIGHT = 600
IMG_SIZE = 600
CHANNELS = 3
N_CLASSES = 5

n_models = 5

# Utils

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=CHANNELS)
    image = (tf.cast(image, tf.float32) / 255.0)
    image = tf.image.resize(image, [ORIGINAL_HEIGHT, ORIGINAL_WIDTH])
    image = tf.reshape(image, [ORIGINAL_HEIGHT, ORIGINAL_WIDTH , CHANNELS])
    return image

def normalize(x):
    x = tf.image.resize(x, [ORIGINAL_HEIGHT, ORIGINAL_WIDTH])
    x = tf.reshape(x, [ORIGINAL_HEIGHT, ORIGINAL_WIDTH, CHANNELS])
    return x

In [ ]:
def read_tfrecord(example):
    TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string), 
        'target': tf.io.FixedLenFeature([], tf.int64), 
    }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    return decode_image(example['image']), tf.cast(example['target'], tf.int32)

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda x: read_tfrecord(x), num_parallel_calls=AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

Data augmentation

In [ ]:
@tf.function
def data_aug(x: tf.Tensor) -> tf.Tensor:
    print("lol",x, "lol")
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.2, lambda: tf.image.random_crop(x, [int(ORIGINAL_HEIGHT*0.8), int(ORIGINAL_WIDTH*0.8), 3]), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.1, lambda: tf.image.random_flip_left_right(x), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.1, lambda: tf.image.random_flip_up_down(x), lambda: x)
    
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.7, lambda: tf.image.random_saturation(x, 0.6, 1.6), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.7, lambda: tf.image.random_brightness(x, 0.05), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.7, lambda: tf.image.random_contrast(x, 0.7, 1.3), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)), lambda: x)
    x = normalize(x)
    print("lol2",x, "lol2")
    return x

In [ ]:
DATASET_FILES = tf.io.gfile.glob(data_path + '/train_tfrecords/ld_train*.tfrec')
dataset_size = count_data_items(DATASET_FILES)
dataset = load_dataset(DATASET_FILES)
dataset = dataset.shuffle(2048).cache()
split = int(dataset_size * 0.2)
train_size = int(dataset_size * 0.8)

print(F"Dataset size: {dataset_size}, split: {split}")

# N fold cross dataset

In [ ]:
!pip install efficientnet

In [ ]:
def n_fold_dataset(augment = True, validation=True, train=True, index=0):
    validation_start = split * index
    validation_end = validation_start + split
    if validation:
        validation_dataset = dataset.skip(validation_start)
        validation_dataset = validation_dataset.take(split)
        validation_dataset = validation_dataset.map(lambda x,y: (normalize(x), y))
        validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(AUTO).cache().repeat()
    
    if train:
        train_dataset_1 = dataset.take(validation_start)
        train_dataset_2 = dataset.skip(validation_end)
        train_dataset_2 = train_dataset_2.take(dataset_size - validation_end)
        train_dataset = train_dataset_1.concatenate(train_dataset_2).cache()
        train_dataset = train_dataset.map(lambda x, y: (data_aug(x), y), num_parallel_calls=AUTO)
        train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTO).repeat()
    
    if not train:
        return validation_dataset
    if not validation:
        return train_dataset
    return train_dataset, validation_dataset

# Loading transfert learning EfficientNet model

In [ ]:
models = []
with strategy.scope():
    for i in range(n_models):
        inputs = layers.Input(shape=(ORIGINAL_HEIGHT, ORIGINAL_WIDTH, 3))
        
        model = Sequential([
            EfficientNet(include_top=False,weights='imagenet', input_tensor=inputs, drop_connect_rate=0.3),
            layers.GlobalAveragePooling2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(5, activation="softmax")
        ])
        model.compile(loss=losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(lr=0.001), metrics=['accuracy'])
        models.append(model)
        
models[0].summary()

In [ ]:
def get_callbacks(i):   
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                              verbose=1, mode='auto', min_delta=0.00001,
                              cooldown=0, min_lr=0)

    early_stopping = EarlyStopping(monitor="val_loss", patience = 8 , verbose = 1, restore_best_weights = True)
    model_cp = ModelCheckpoint(F'EfficientNetB3_tl_best_weights_{i}.h5', 
                                 save_best_only = True, 
                                 save_weights_only = True,
                                 monitor = 'val_loss', 
                                 mode = 'min', verbose = 1)
    
    return [early_stopping, model_cp, reduce_lr]

In [ ]:
histories = []
for i, model in enumerate(models):
    print(F"Training model: {i}")
    train_dataset, validation_dataset = n_fold_dataset(augment = True, index=i)

    history = model.fit(train_dataset,
          steps_per_epoch=train_size//BATCH_SIZE,
          validation_steps=split//BATCH_SIZE,
          validation_data=validation_dataset,
          batch_size=BATCH_SIZE, epochs=100, 
          callbacks=get_callbacks(i))
    histories.append(history)
    gc.collect()

In [ ]:
accuracies = []
for i in range(n_models):
    validation_dataset = n_fold_dataset(train=False, index=i)
    accuracies.append(model.evaluate(validation_dataset, steps=split//BATCH_SIZE)[1])
        
print("Accuracy:", sum(accuracies) / len(accuracies) )

# Plot all models

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(16, 12))

for i, history in enumerate(histories):
    history_df = pd.DataFrame(history.history)
    history_df[['loss', 'val_loss']].plot(ax=axes[i,0])
    history_df[['accuracy', 'val_accuracy']].plot(ax=axes[i,1])